# Complete Active Learning Optimisation Notebook


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/digiLab-ai/ue-sdk-helper/blob/main/notebooks/05_active_learning_workflow.ipynb)

In this notebook, it is demonstrated how to use the active learning functions that have been deployed in this repository.



## 1. Setup

Firstly import the relevant Python libaries and the helper functions from ue_helper and data_generation

In [ ]:

# =============================================================================
# 1. IMPORTS
# =============================================================================

import sys
import os

# Add the src directory to Python path so we can import ue_helper and data_generation
current_dir = os.getcwd()
if 'notebooks' in current_dir:
    src_path = os.path.join(current_dir, '..', 'src')
else:
    src_path = os.path.join(current_dir, 'src')

sys.path.insert(0, src_path)
import numpy as np
import pandas as pd
import requests
from io import StringIO
from datetime import datetime
from typing import Callable, List, Tuple, Dict, Any

# Uncertainty Engine imports
from uncertainty_engine import Client
from uncertainty_engine.graph import Graph
from uncertainty_engine.nodes.base import Node
from uncertainty_engine_types import ResourceID
from uncertainty_engine.nodes.workflow import Workflow

# Import active learning functions from the ue_helper package
from ue_helper import active_learning_step, active_learning_loop, get_project_id

# Import function generators and sampling utilities from data_generation package
from data_generation import (
    generate_branin_function,
    latin_hypercube_sampling,
    FunctionInfo
)

print("✅ All libraries imported successfully!")




Next, run the client setup, returning client, project_id and projects_dict


N.B. This method requires implementaiton of password and username as in the previous workbooks when deployed on Google Colab.

In [ ]:

# =============================================================================
# 2. CLIENT SETUP (USING YOUR CREDENTIALS)
# =============================================================================

def setup_client():
    """Setup and authenticate the Uncertainty Engine client using your credentials."""
    print("🔧 Setting up Uncertainty Engine client...")
    
    # Create a userdata-like dictionary (same as your working version)
 
    
    client.authenticate(os.environ["UE_ACCOUNT_ID"])


    # Get projects dictionary (same as your working version)
    projects_dict = {proj.name: proj.id for proj in client.projects.list_projects()}
    project_id = projects_dict["Personal"]  # Using "Personal" project like your working version
    
    print(f"✅ Client authenticated successfully!")
    print(f"✅ Available projects: {list(projects_dict.keys())}")
    
    return client, project_id, projects_dict



Now that our client is set up, we can define a workflow. Here, we call the pre-generated functions in our complete_workflow_example function (which calls upon the Branin Generator and the active learning loop helper functions). It is then exectured in if __name__ == "__main__":

In [10]:
def complete_workflow_example():
    """Complete example using the Branin function - same as your working version."""
    
    print("=" * 60)
    print("COMPLETE ACTIVE LEARNING WORKFLOW EXAMPLE")
    print("=" * 60)
    
    # Step 1: Generate the Branin function using the imported generator
    branin_info = generate_branin_function()
    print(f"Optimizing: {branin_info.name}")
    print(f"Description: {branin_info.description}")
    print(f"Global minimum: {branin_info.global_minimum}")
    print(f"Global minimum points: {branin_info.global_minimum_points}")
    
    # Step 2: Generate initial data using Latin Hypercube Sampling (same as your version)
    np.random.seed(42)  # Same seed as your working version
    n_initial = 8  # Same number of initial samples as your working version
    initial_samples = latin_hypercube_sampling(branin_info.search_space, n_initial)
    initial_input_data = initial_samples.tolist()
    initial_output_data = [[branin_info.function(point)] for point in initial_input_data]
    
    print(f"\nInitial data points:")
    for i, (input_here, output) in enumerate(zip(initial_input_data, initial_output_data)):
        print(f"  Point {i+1}: ({input_here[0]:.3f}, {input_here[1]:.3f}) -> {output[0]:.3f}")
    
    # Step 3: Setup client and project
    try:
        client, project_id, projects_dict = setup_client()
        
        # Step 4: Run active learning (same parameters as your working version)
        print(f"\n🚀 Starting active learning optimization...")
        results = active_learning_loop(
            client=client,
            project_id=project_id,
            initial_input_data=initial_input_data,
            initial_output_data=initial_output_data,
            objective_function=branin_info.function,
            num_iterations=8,  # Same as your working version
            acquisition_function="PosteriorStandardDeviation",
            num_points_per_iteration=1
        )
        
        # Step 5: Analyze results
        print(f"\n=== RESULTS ===")
        print(f"Best point found: {results['best_point']['coordinates']}")
        print(f"Best value: {results['best_point']['value']:.6f}")
        print(f"Convergence to global minimum: {abs(results['best_point']['value'] - branin_info.global_minimum):.6f}")
        print(f"Total evaluations: {len(results['final_input_data'])}")
        
        # Show convergence history
        print(f"\nConvergence history:")
        for i, best_val in enumerate(results['convergence_history']):
            print(f"  Iteration {i+1}: {best_val:.6f}")
        
        # Show all recommendations
        print(f"\nAll recommendations:")
        for rec in results['all_recommendations']:
            print(f"  Iteration {rec['iteration']}: {rec['coordinates']} -> {rec['value']:.6f}")
        
        return results
        
    except Exception as e:
        print(f"❌ Error during active learning: {e}")
        print(f"Please check your client setup and try again.")
        return None


if __name__ == "__main__":
    print("🚀 Starting Active Learning Workflow Script")
    print("=" * 50)
    
    # Run examples
    print("\n2. Running complete workflow example...")
    complete_workflow_example()
    
    print("\n" + "=" * 50)
    print("✅ Script completed successfully!")
    print("\nFor support, contact:")
    print("- dan.greenhouse@digilab.ai")
    print("- Help Center: get.support.uncertaintyengine.ai")


🚀 Starting Active Learning Workflow Script

2. Running complete workflow example...
COMPLETE ACTIVE LEARNING WORKFLOW EXAMPLE
Optimizing: Branin
Description: 2D test function with 3 global minima. Domain: x1 in [-5, 10], x2 in [0, 15]
Global minimum: 0.397887
Global minimum points: [[-3.141592653589793, 12.275], [3.141592653589793, 2.275], [9.42478, 2.475]]

Initial data points:
  Point 1: (-4.298, 0.001) -> 237.923
  Point 2: (6.359, 3.735) -> 26.503
  Point 3: (9.749, 7.513) -> 23.471
  Point 4: (-1.342, 9.418) -> 13.274
  Point 5: (2.793, 4.908) -> 6.475
  Point 6: (4.667, 12.234) -> 127.251
  Point 7: (0.122, 6.772) -> 20.461
  Point 8: (1.747, 13.875) -> 113.610
🔧 Setting up Uncertainty Engine client...
✅ Client authenticated successfully!
✅ Project ID: 
✅ Available projects: ['Personal']

🚀 Starting active learning optimization...
Starting active learning loop with 8 iterations...
Initial data points: 8

--- Iteration 1/8 ---
  Added point: (9.7491, 13.8747) -> 124.3882
  Current

KeyboardInterrupt: 

As seen in the outputs, the active learning loop runs for several iterations, updating its findings on while optimising with the Branin test function as its objective. 

